<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span></li><li><span><a href="#Reading-in-csv-Data-as-Dataframes" data-toc-modified-id="Reading-in-csv-Data-as-Dataframes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reading in csv Data as Dataframes</a></span><ul class="toc-item"><li><span><a href="#Merge-and-Save-Data-to-Data-Dir" data-toc-modified-id="Merge-and-Save-Data-to-Data-Dir-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Merge and Save Data to Data Dir</a></span></li></ul></li><li><span><a href="#Retrieve-Stellar-Radii-from-Gaia" data-toc-modified-id="Retrieve-Stellar-Radii-from-Gaia-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Retrieve Stellar Radii from Gaia</a></span><ul class="toc-item"><li><span><a href="#Parse-Target-Coordinates-into-SkyCoord-Objects" data-toc-modified-id="Parse-Target-Coordinates-into-SkyCoord-Objects-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Parse Target Coordinates into SkyCoord Objects</a></span></li><li><span><a href="#Query-Gaia-Using-Astroquery" data-toc-modified-id="Query-Gaia-Using-Astroquery-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Query Gaia Using Astroquery</a></span></li></ul></li></ul></div>

Kaimi Kahihikolo

**Updated** : 2018-06-18 03:34:01 

Below is a script to read in and merge the csv data files.

## Preamble

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
main_dir = "/Volumes/LaCie/Research/NAC/"
data_dir = "{}data/".format(main_dir)
out_dir = "{}output/".format(main_dir)

## Reading in csv Data as Dataframes

In [3]:
df1 = pd.read_csv("{}MN_17_3616_MJ_Table2.csv".format(data_dir))

df2 = pd.read_csv("{}MN_17_3616_MJ_Table4.csv".format(data_dir))

The csv files have a second header (with units) that is causing problems, so I will just remove it.

In [4]:
df1 = df1.drop(0)
df2 = df2.drop(0)
df2 = df2.drop(columns=["PERIOD", "Name"])

In [5]:
df2.columns

Index(['MK', 'logL/Lo', 'SNR', 'RV', 'SRV', 'FWHM', 'SFWHM', 'FWHMo', 'VSINI',
       'EVSINI', 'Rsini'],
      dtype='object')

In [6]:
df1.index = range(len(df1))
df2.index = range(len(df1))

### Merge and Save Data to Data Dir

In [7]:
merge = pd.merge(df1, df2, left_index=True,
                right_index=True, how="outer",
                suffixes=("", "_y"))

merge.to_csv("{}MN_17_3616_merge.csv".format(out_dir),
            float_format='%g')

## Retrieve Stellar Radii from Gaia

Below I will query the [Gaia Archive](https://gea.esac.esa.int/archive/) to retrive the stellar radius, *radius_val*

In [8]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord

### Parse Target Coordinates into SkyCoord Objects

In [9]:
def str_merge(str1, str2):
    """
    Parameters
    ----------
        str1 : str 
            First string
        str2 : (str)
            Second string
    Returns
    -------
        (str) str1 and str2 separated by a " "
    """
    return "{} {}".format(str1, str2)

In [10]:
def to_skycoord(coords):
    """
    Parameters
    ----------
    coords : (list)
            An object RA and DEC in (hourangle, deg) format
    Returns
    -------
    c : SkyCoord
        Object coordinates mapped to a SkyCoord object
    """
    coord = str_merge(*coords)
    c = SkyCoord(coord, unit=(u.hourangle, u.deg))
    return c

In [11]:
coords = merge[["RA", "Dec"]].apply(to_skycoord, axis=1)

### Query Gaia Using Astroquery

To query the Gaia database, we will be using the [Astroquery library](http://astroquery.readthedocs.io/en/latest/gaia/gaia.html). 
```Python
from astroquery.gaia import Gaia
```

For example,

```python
job = Gaia.launch_job_async("SELECT * FROM gaiadr1.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr1.gaia_source.ra, gaiadr1.gaia_source.dec),CIRCLE('ICRS',56.75,24.1167,2))=1;", dump_to_file=True)

r = job.get_results()
print (r['source_id'])
```
Returns and astropy table of the form:

```text
source_id    
-----------------
66926207631181184
66818318054203520
66917823855519360
66830859358837888
...
```

For a list of tables, use...
```python
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())
```

Other [tutorial](https://gea.esac.esa.int/archive-help/tutorials/python_cluster/index.html)

In [12]:
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [13]:
width = u.Quantity(2.0, u.arcsec)
height = u.Quantity(2.0, u.arcsec)

In [14]:
from multiprocessing import Manager, Pool, active_children

In [15]:
def arg_nearest_vector(array, value):
    array = np.asarray(array)
    
    if len(array) == 1:
        return 0
    else:
        idx = np.array([np.linalg.norm(x+y) for (x,y) in array-value]).argmin()
        return idx

In [16]:
N = len(coords)

## -- List of keys to retrieve from Gaia database
features = ["parallax", "parallax_error",
            "radius_val", "radius_percentile_lower", "radius_percentile_upper",
            "lum_val", "lum_percentile_lower", "lum_percentile_upper",
            "teff_val", "teff_percentile_lower", "teff_percentile_upper"]

mylist = Manager().list()

def main(n):
    global features
    coord = n
    
    col = np.ones(len(features)+1)
    
    if coord % 10 == 0:
        print("{} / {}".format(coord, N))
    
    try:
        r = Gaia.query_object_async(
                coordinate=coords.values[coord], 
                width = width,
                height = height
            );
        cat = SkyCoord(ra=r["ra"].data*u.deg, dec=r["dec"].data*u.deg)
        #print(cat)
        
        c = SkyCoord(coords.loc[coord])
        
        idx, _, _ = c.match_to_catalog_sky(cat)
        
        idx = int(idx)
        
#         idx = arg_nearest_vector([r["ra"].data,
#                            r["dec"].data], 
#                       [coords.values[coord].ra.deg, 
#                        coords.values[coord].dec.deg])
        
        col[0] = coord
        col[1] = float((r["parallax"]).data[idx])
        col[2] = float((r["parallax_error"]).data[idx])
        col[3] = float((r["radius_val"]).data[idx])
        col[4] = float((r["radius_percentile_lower"]).data[idx])
        col[5] = float((r["radius_percentile_upper"]).data[idx])
        col[6] = float((r["lum_val"]).data[idx])
        col[7] = float((r["lum_percentile_lower"]).data[idx])
        col[8] = float((r["lum_percentile_upper"]).data[idx])
        col[9] = float((r["teff_val"]).data[idx])
        col[10] = float((r["teff_percentile_lower"]).data[idx])
        col[11] = float((r["teff_percentile_upper"]).data[idx])


    except TypeError:
        col[0] = coord
        col[1] = float((r["parallax"]).data[0])
        col[2] = float((r["parallax_error"]).data[0])
        col[3] = float((r["radius_val"]).data[0])
        col[4] = float((r["radius_percentile_lower"]).data[0])
        col[5] = float((r["radius_percentile_upper"]).data[0])
        col[6] = float((r["lum_val"]).data[0])
        col[7] = float((r["lum_percentile_lower"]).data[0])
        col[8] = float((r["lum_percentile_upper"]).data[0])
        col[9] = float((r["teff_val"]).data[0])
        col[10] = float((r["teff_percentile_lower"]).data[0])
        col[11] = float((r["teff_percentile_upper"]).data[0])
    
    mylist.append(col)

In [17]:
from time import sleep
if __name__ == "__main__":
    pool = Pool(10)
    
    for i in range(N):
        pool.apply_async(main, (i,))
    pool.close()
    
    while len(active_children()) > 1:
        sleep(0.5)
    pool.join()

0 / 319
Query finished.
Query finished.
Query finished.
10 / 319
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
20 / 319
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
30 / 319
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
40 / 319
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
50 / 319
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
60 / 319
Query finished.
Query finished.
Query finished.
Query finished.
Query fini

In [18]:
df = pd.DataFrame([i for i in mylist])
df.columns = ["temp_idx"]+features

In [19]:
df = df.sort_values(by="temp_idx").drop(columns="temp_idx")
df.index = range(len(df))

In [20]:
df["radius_sigma"] = pd.DataFrame([df["radius_percentile_upper"] - df["radius_val"],
                                 df["radius_val"] - df["radius_percentile_lower"]]).mean(axis=0)

df["lum_sigma"] = pd.DataFrame([df["lum_percentile_upper"] - df["lum_val"],
                                 df["lum_val"] - df["lum_percentile_lower"]]).mean(axis=0)

df["teff_sigma"] = pd.DataFrame([df["teff_percentile_upper"] - df["teff_val"],
                                 df["teff_val"] - df["teff_percentile_lower"]]).mean(axis=0)

In [24]:
merge1 = pd.merge(merge.reindex(), df.reindex(), left_index=True,
                right_index=True, how="outer",
                suffixes=("", "_y"))

merge1.to_csv("{}MN_17_3616_gaia.csv".format(out_dir),
              float_format='%g')